# Workflow to verify `NHD+ HR` dataset

## Importing necessary libraries

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx

import fiona

from typing import (
    Dict,
    Tuple,
    Union,
)

In [2]:
# defining paths
# downloaded from https://www.sciencebase.gov/catalog/item/5d967365e4b0c4f70d113923
tgf_path = '/Users/kasrakeshavarz/Documents/geospatial-data/StMary-Milk/TGF/TGF.gdb/'

# list tgf layers
fiona.listlayers(tgf_path)

['POIs', 'waterbodies', 'nhru', 'nsegment']

In [3]:
# read tgf layers
## river segments
tgf_riv = gpd.read_file(tgf_path, driver='FileGDB', layer='nsegment')
## basin segments
tgf_cat = gpd.read_file(tgf_path, driver='FileGDB', layer='nhru')
## Points of Interest (POIs)
tgf_poi = gpd.read_file(tgf_path, driver='FileGDB', layer='POIs')

## Necessary functions

In [4]:
# Checking wether spatial connectivities are valid

def spatial_conn(
    gdf: gpd.GeoDataFrame,
    main_id: str=None,
    ds_main_id: str=None,
    print_err: bool=True,
) -> Tuple[Dict, Dict]:
    
    '''Validates downstream IDs and checks spatial connectivity 
    of river elements
    
    Parameters:
    -----------
    gdf: geopandas.GeoDataFrame
        a GeoPandas dataframe containing details of a river network
        with minimum details of element ID, downstream element ID, and
        geometries
    main_id: str, defaults to `None`
        String defining the column of element IDs in the input geopandas
        dataframe
    ds_main_id: str, defaults to `None`
        String defining the column of downstream element IDs in the
        input geopandas dataframe
    print_err: bool, defaults to `True` (optional)
        Argument to specify either print the detailed errors within the
        input `gdf`
        
    Returns:
    --------
    connections: Dict
        A dictionary of connections with keys as `main_id` and
        values of `ds_main_id` if no issues were found
    wrong_conns: Dict
        A dictionary of problematic connections with keys as `main_id`
        and values defining the nature of the problem, available error
        types: 1) spatial disconnection (`spdis`), 2) missing downstream
        element (misds)
        
    '''
    
    # necessary initializations
    connections = {} # key: `main_id`, value: `ds_main_id`
    wrong_conns = {} # key: `main_id`, value: error type
    
    
    if main_id is None:
        raise ValueError("main_id cannot be `None`")
        
    if ds_main_id is None:
        raise ValueError("ds_main_id cannot be `None`")

    # Loop through each feature in the shapefile
    for idx, feature in gdf.iterrows():

        # Skip over features with DS_Main_ID = 0
        if feature[ds_main_id] == 0:
            continue

        # Find the corresponding descendant feature
        descendant = riv.loc[riv[main_id] == feature[ds_main_id]]

        # Check if the descendant DataFrame is empty
        if descendant.empty:
            # find possible downstream IDs of 'DS_Main_ID' does not exist
            possible_ds = gdf.geometry.intersects(gdf.loc[idx].geometry)
            possible_ds_list = gdf[possible_ds][main_id].tolist()
            
            # add the `Main_ID` to the `wrong_conns` list
            wrong_conns[feature[main_id]] = 'misds'

            # print details
            if print_err:
                print(f"Warning: No descendant feature found for Main_ID {feature[main_id]} with DS_Main_ID {feature[ds_main_id]}")
                print(f"  Possible downstream IDs are: {possible_ds_list}")

            continue

        # Check if the feature's geometry intersects with the descendant's geometry
        if not feature.geometry.intersects(descendant.geometry.iloc[0]):

            if print_err:
                print(f"Feature with Main_ID {feature['Main_ID']} is not spatially connected to its descendant with DS_Main_ID {feature['DS_Main_ID']}")
                
            # add the `Main_ID` to the `wrong_conns` dictionary
            wrong_conns[feature['Main_ID']] = 'spdis'
        else:
            # add the `Main_ID` to the `connections` dictionary
            connections[feature["Main_ID"]] = feature["DS_Main_ID"]
            
    return connections, wrong_conns

In [5]:
def find_cycles(
    gdf: gpd.GeoDataFrame,
    main_id: str,
    ds_main_id: str,
    l: int=None,
    print_err: bool=True,
) -> list[...]:
    '''Returns the cycles of length `l` of a river network given 
    as a geopandas GeoDataFrame
    
    Parameters:
    -----------
    gdf: geopandas.GeoDataFrame
        GeoDataFrame of river segments including at least three pieces
        of information: 1) geometries of segments, 2) segment IDs, and
        3) downstream segment IDs
    main_id: str, defaults to `None`
        String defining the column of element IDs in the input geopandas
        dataframe
    ds_main_id: str, defaults to `None`
        String defining the column of downstream element IDs in the
        input geopandas dataframe
    l: int, defaults to None (optional)
        length of cycles to be found, leave as None to find
        cycles of any length
    print_err: bool, defaults to `True` (optional)
        print cycles if stated
    
    Returns:
    --------
    cycles: list
        List of `main_id`s indicating cycles
    
    '''
    
    # creating a DiGraph out of `gdf` object
    riv_graph = nx.from_pandas_edgelist(gdf, source=main_id, target=ds_main_id, create_using=nx.DiGraph)

    # extracting cycles
    cycles = list(nx.simple_cycles(riv_graph, l))
    
    if print_err:
        print(cycles)
    
    return cycles

In [6]:
def find_upstream(
    gdf: gpd.GeoDataFrame,
    target_id: Union[str, int, ...],
    main_id: str,
    ds_main_id: str,
) -> set[...]:
    '''Find "ancestors" or upstream segments in a river network given
    in the from of a geopandas.GeoDataFrame `gdf`
    
    Parameters:
    -----------
    gdf: geopandas.GeoDataFrame
        GeoDataFrame of river segments including at least three pieces
        of information: 1) geometries of segments, 2) segment IDs, and
        3) downstream segment IDs
    target_id: str, int, or any other data type as included in `gdf`
        Indicating the target ID anscestor or upstream of which is
        desired
    main_id: str
        String defining the column of element IDs in the input geopandas
        dataframe
    ds_main_id: str
        String defining the column of downstream element IDs in the
        input geopandas dataframe
    
    Returns:
    --------
    nodes: list
        IDs of nodes being upstream or anscestor of the `target_id`
    
    '''
    # creating a DiGraph out of `gdf` object
    riv_graph = nx.from_pandas_edgelist(gdf, source=main_id, target=ds_main_id, create_using=nx.DiGraph)
    
    # return nodes in a list
    nodes = nx.ancestors(riv_graph, target_id)
    
    return nodes

## Modifiying the river segments

### Correcting inaccurate `Main_ID` and `DS_Main_ID` values

In [7]:
# safe copy of `tgf_riv` for further analysis
riv = tgf_riv.copy()

In [8]:
# list of columns to be chosen for dtype downcast
col_list = ['Main_ID', 'DS_Main_ID']

# downcast dtype of necessary columns to integer, if possible
fcols = riv[col_list].select_dtypes('float').columns
riv[fcols] = riv[fcols].apply(pd.to_numeric, downcast='integer')

/Users/kasrakeshavarz/Documents/virtualenvs/jupyter/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:380: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)


In [9]:
# checking if downcasting is done right
for col in col_list:
    print(f'Downcasting for the dtype of column `{col}` is right: ', (tgf_riv[col] == riv[col]).astype(int).sum() == tgf_riv.shape[0])

Downcasting for the dtype of column `Main_ID` is right:  True
Downcasting for the dtype of column `DS_Main_ID` is right:  True


### Check if each element (`Main_ID`) is spatially connected to its `DS_Main_ID`

In [10]:
conns, err_conns = spatial_conn(riv, 'Main_ID', 'DS_Main_ID', True)

Feature with Main_ID 65000100001397 is not spatially connected to its descendant with DS_Main_ID 65000100007950
Feature with Main_ID 65000100004140 is not spatially connected to its descendant with DS_Main_ID 65000100003646
Feature with Main_ID 65000200011511 is not spatially connected to its descendant with DS_Main_ID 65000200007422
Feature with Main_ID 65000300007677 is not spatially connected to its descendant with DS_Main_ID 65000300008719
Feature with Main_ID 65000300007818 is not spatially connected to its descendant with DS_Main_ID 65000300008719
Feature with Main_ID 55001100010923 is not spatially connected to its descendant with DS_Main_ID 55001100001643
Feature with Main_ID 55001100031439 is not spatially connected to its descendant with DS_Main_ID 55001100001635
Feature with Main_ID 55001100063905 is not spatially connected to its descendant with DS_Main_ID 55001100029740
Feature with Main_ID 55000500001843 is not spatially connected to its descendant with DS_Main_ID 5500050

### Checking existing directed cycles

River networks could be considered a special case of directed acyclic graphs (DAGs), therefore, for verification purposes of this Notebook, we could use algorithms designed for this specific type of graphs. `NetworkX` usually has necessary algorithms for graph analysis!

In [11]:
cycles = find_cycles(riv, 'Main_ID', 'DS_Main_ID')

[[65000300006184], [55000500004665, 55000500004677]]


### Fixing irregular river segment IDs

#### Invalid downstream IDs

Visually, valid IDs are evaluated and assigned to those reported above. If the downstream/upstream element connectivity detail is correct, but spatial disconnectivity is observed, no correction is needed as data are fed into routing models for hydrology, and spatial connectivity does not matter.

In [12]:
# for the segment ID of `err_conns`, the correct downstream IDs are assigned
misds_corr = {
    55000800013451: 55000800039418,
    55000800016316: 55000800013888,
    55000800017535: 55000800016193,
    55000800020554: 55000800013888,
    55000800032134: 55000800030365,
}

for us, ds in misds_corr.items():
    riv.loc[riv['Main_ID'] == us, 'DS_Main_ID'] = ds

In [13]:
conns, err_conns = spatial_conn(riv, 'Main_ID', 'DS_Main_ID', True)

Feature with Main_ID 65000100001397 is not spatially connected to its descendant with DS_Main_ID 65000100007950
Feature with Main_ID 65000100004140 is not spatially connected to its descendant with DS_Main_ID 65000100003646
Feature with Main_ID 65000200011511 is not spatially connected to its descendant with DS_Main_ID 65000200007422
Feature with Main_ID 65000300007677 is not spatially connected to its descendant with DS_Main_ID 65000300008719
Feature with Main_ID 65000300007818 is not spatially connected to its descendant with DS_Main_ID 65000300008719
Feature with Main_ID 55001100010923 is not spatially connected to its descendant with DS_Main_ID 55001100001643
Feature with Main_ID 55001100031439 is not spatially connected to its descendant with DS_Main_ID 55001100001635
Feature with Main_ID 55001100063905 is not spatially connected to its descendant with DS_Main_ID 55001100029740
Feature with Main_ID 55000500001843 is not spatially connected to its descendant with DS_Main_ID 5500050

#### Cycles

Iteratively, the cycles are corrected manually. I cannot think of any automated way to correct the cycles. Feel free to let me know if you are aware of any!

In [14]:
# Iteration 1
# cycles found are corrected by assigning the right downstream IDs inspected visually

cycles_corr = {
    65000300006184: 65000300005651,
    55000500004677: 55000500004714,
}

for us, ds in cycles_corr.items():
    riv.loc[riv['Main_ID'] == us, 'DS_Main_ID'] = ds
    
cycles = find_cycles(riv, 'Main_ID', 'DS_Main_ID')

[[55000500004714, 55000500004677]]


In [15]:
# Iteration 2
# cycles found are corrected by assigning the right downstream IDs inspected visually

cycles_corr[55000500004714] = 55000500004576

for us, ds in cycles_corr.items():
    riv.loc[riv['Main_ID'] == us, 'DS_Main_ID'] = ds
    
cycles = find_cycles(riv, 'Main_ID', 'DS_Main_ID')

[]


## Dissolving left-/rightbanks of sub-basins

For `dissolving` the sub-basins with identical IDs, the `hru_segment_nhm` attribute has been chosen as the main ID. Upon evaluations of the 

In [59]:
cat

,POI_ID,hru_id,hru_segment,hru_id_tb,hru_segment_tb,hru_id_nhm,hru_segment_nhm,Type_NCA,HUC04,Coastal_HRU,Shape_Length,Shape_Area,geometry
0,5.000100e+12,1,115,1,115,109952,56575,0,0101,0,68572.964172,5.598540e+07,"MULTIPOLYGON (((1965355.000 2853505.000, 19653..."
1,5.000100e+12,2,115,2,115,109953,56575,0,0101,0,77310.884173,7.214900e+07,"MULTIPOLYGON (((1965255.000 2853515.000, 19652..."
2,5.000100e+12,3,102,3,102,109954,56562,0,0101,0,114903.231027,1.221173e+08,"MULTIPOLYGON (((1998635.000 2855765.000, 19989..."
3,5.000100e+12,4,102,4,102,109955,56562,0,0101,0,77169.650027,5.707688e+07,"MULTIPOLYGON (((1995185.000 2855915.000, 19951..."
4,5.000100e+12,5,83,5,83,109956,56543,0,0101,0,10189.003249,4.672174e+06,"MULTIPOLYGON (((2168483.261 2873639.810, 21684..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12158,5.500080e+13,2748,0,12159,0,122110,0,0,1711,1,97979.999801,4.467460e+07,"MULTIPOLYGON (((-1965705.000 3111385.000, -196..."
12159,5.500080e+13,2749,0,12160,0,122111,0,0,1711,1,29939.999800,2.136320e+07,"MULTIPOLYGON (((-1960875.000 3119755.000, -196..."
12160,5.500080e+13,2750,558,12161,5197,122112,61657,0,1711,0,89608.432529,8.206560e+07,"MULTIPOLYGON (((-1953402.339 3174948.721, -195..."
12161,5.500080e+13,2751,0,12162,0,122113,0,0,1711,1,114200.010001,9.240170e+07,"MULTIPOLYGON (((-1956185.000 3164605.000, -195..."


## Correcting river segments without a corresponding sub-basin ID (nhru)

## Extracting river segments of the `SMM` area based on upstream/downstream connectivity

Based on the boundary of the HUC12 dataset shared by [Jamie Kolodinsky](jamie.kolodinsky@ec.gc.ca), for the St. Mary portion, the most downstream river segment is `65000400000239`, and for the Milk portion is `23002800001207`.

In [16]:
# empty dictionary for extracting ancestor nodes
nodes = {}

# dictionary of target ids
target_ids = {
    'st_mary': 65000400000239,
    'milk': 23002800001207,
}

# extracting ancestors
for river, target in target_ids.items():
    # extracting ancestors
    ancestors = find_upstream(riv, target, 'Main_ID', 'DS_Main_ID')
    # adding target node to the `ancestors` set
    ancestors.add(target)
    # adding to the ancestor dictionary
    nodes[river] = ancestors

## Extracting river segments of the `SMM` area based on spatial overlap with the pre-defined boundary

## Final verifications

____

In [77]:
freq_count = cat.hru_segment_nhm.value_counts()
f = freq_count.sort_values()

In [80]:
f[f>2]

61398      3
60122      3
59982      3
57791      3
61228      3
        ... 
60270      4
60006      4
59474      5
57444      5
0        565
Name: hru_segment_nhm, Length: 93, dtype: int64

In [66]:
filtered_cat = cat[cat['hru_segment_nhm'] != 0]

In [67]:
d = filtered_cat.dissolve(by='POI_ID', as_index=False)

In [68]:
# Get polygons with zero 'hru_segment' values
zero_values_gdf = cat[cat['hru_segment_nhm'] == 0]

# Append polygons with zero 'hru_segment' values to the dissolved GeoDataFrame
final_gdf = gpd.GeoDataFrame(pd.concat([d, zero_values_gdf]))

In [69]:
d

,POI_ID,geometry,hru_id,hru_segment,hru_id_tb,hru_segment_tb,hru_id_nhm,hru_segment_nhm,Type_NCA,HUC04,Coastal_HRU,Shape_Length,Shape_Area
0,5.000100e+12,"POLYGON ((2243155.000 2905805.000, 2243145.000...",66,1,66,1,110017,56461,0,0101,0,25880.003500,1.329690e+07
1,5.000100e+12,"POLYGON ((2241435.000 2902295.000, 2241445.000...",53,2,53,2,110004,56462,0,0101,0,57756.700675,8.006604e+07
2,5.000100e+12,"POLYGON ((2227975.000 2900555.000, 2227975.000...",46,3,46,3,109997,56463,0,0101,0,21188.830216,3.240654e+06
3,5.000100e+12,"MULTIPOLYGON (((2229045.000 2897345.000, 22290...",41,4,41,4,109992,56464,0,0101,0,184064.496781,3.333918e+08
4,5.000100e+12,"MULTIPOLYGON (((2198495.000 2891445.000, 21985...",27,5,27,5,109978,56465,0,0101,0,145703.907164,2.449532e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5802,6.500040e+13,"POLYGON ((-1307965.000 2985175.000, -1307973.0...",5,57,3543,1777,113494,58237,0,0904,0,23397.478246,1.271348e+07
5803,6.500040e+13,"POLYGON ((-1292815.000 3054565.000, -1292815.0...",86,58,3624,1778,113575,58238,0,0904,0,107065.656064,7.048516e+07
5804,6.500040e+13,"POLYGON ((-1316195.000 3045775.000, -1316205.0...",73,59,3611,1779,113562,58239,0,0904,0,44926.283398,2.182766e+07
5805,6.500040e+13,"POLYGON ((-1252665.000 3056795.000, -1252675.0...",93,60,3631,1780,113582,58240,0,0904,0,43697.357486,1.207924e+07


In [70]:
final_gdf

,POI_ID,geometry,hru_id,hru_segment,hru_id_tb,hru_segment_tb,hru_id_nhm,hru_segment_nhm,Type_NCA,HUC04,Coastal_HRU,Shape_Length,Shape_Area
0,5.000100e+12,"POLYGON ((2243155.000 2905805.000, 2243145.000...",66,1,66,1,110017,56461,0,0101,0,25880.003500,1.329690e+07
1,5.000100e+12,"POLYGON ((2241435.000 2902295.000, 2241445.000...",53,2,53,2,110004,56462,0,0101,0,57756.700675,8.006604e+07
2,5.000100e+12,"POLYGON ((2227975.000 2900555.000, 2227975.000...",46,3,46,3,109997,56463,0,0101,0,21188.830216,3.240654e+06
3,5.000100e+12,"MULTIPOLYGON (((2229045.000 2897345.000, 22290...",41,4,41,4,109992,56464,0,0101,0,184064.496781,3.333918e+08
4,5.000100e+12,"MULTIPOLYGON (((2198495.000 2891445.000, 21985...",27,5,27,5,109978,56465,0,0101,0,145703.907164,2.449532e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12156,5.500080e+13,"MULTIPOLYGON (((-1957275.000 3104325.000, -195...",2746,0,12157,0,122108,0,0,1711,1,64520.001701,2.716830e+07
12158,5.500080e+13,"MULTIPOLYGON (((-1965705.000 3111385.000, -196...",2748,0,12159,0,122110,0,0,1711,1,97979.999801,4.467460e+07
12159,5.500080e+13,"MULTIPOLYGON (((-1960875.000 3119755.000, -196...",2749,0,12160,0,122111,0,0,1711,1,29939.999800,2.136320e+07
12161,5.500080e+13,"MULTIPOLYGON (((-1956185.000 3164605.000, -195...",2751,0,12162,0,122113,0,0,1711,1,114200.010001,9.240170e+07


____

In [ ]:
import easymore

# easymore initiation
esmr = easymore.easymore()

# find all upstream river segments for the outlets of the  St. Mary and Milk river basins
seg_ids = riv['Main_ID']
down_seg_ids = riv['DS_Main_ID']
ntopo = esmr.get_all_downstream(seg_ids, down_seg_ids)

In [ ]:
upstream_ids = esmr.get_all_upstream(23002800001207, ntopo).astype(int)

In [ ]:
smm_riv = riv.loc[riv['Main_ID'].isin(upstream_ids), :]

In [ ]:
upstream_stmary = esmr.get_all_upstream(65000400000239, ntopo).astype(int)

In [ ]:
stmary_riv = riv.loc[riv['Main_ID'].isin(upstream_stmary), :]

In [ ]:
len(stmary_riv)

In [ ]:
stmary_riv.to_file('/Users/kasrakeshavarz/Documents/temp/stmary.shp')